In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
Biker = candidate_subclass('Biker', ['person', 'bike'])

In [3]:
user_lists = {
    'spouse':  ['spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'],
    'family':  ['father', 'father', 'mother', 'sister', 'sisters', 
                'brother', 'brothers', 'son', 'sons', 'daughter', 'daughters',
                'grandfather', 'grandmother', 'uncle', 'uncles', 'aunt', 'aunts', 
                'cousin', 'cousins'],
    'other':  ['boyfriend', 'girlfriend', 'boss', 'employee', 'secretary', 'co-worker']}

In [4]:
from snorkel.contrib.babble import SemanticParser
semparser = SemanticParser(
    mode='image', candidate_class=Biker, user_lists=user_lists, beam_width=10)

Created grammar with 431 rules


In [5]:
from snorkel.contrib.babble import Explanation

explanations = [
    Explanation(
        name='LF_foo',
        condition="box X is near box Y",
        candidate='2ca6dbbb-870c-4e34-8053-0ac2dbd850f5::span:798:808~~2ca6dbbb-870c-4e34-8053-0ac2dbd850f5::span:839:851',
        label=True,
        semantics=None),  
#     Explanation(
#         name='LF_spouse_between',
#         condition="there is a spouse word between arg 1 and arg 2",
#         candidate='2ca6dbbb-870c-4e34-8053-0ac2dbd850f5::span:798:808~~2ca6dbbb-870c-4e34-8053-0ac2dbd850f5::span:839:851',
#         label=True,
#         semantics=None),      
#     Explanation(
#         name='LF_no_spouse_in_sentence',
#         condition="there are no spouse words in the sentence",
#         candidate='d0de6a86-66d5-40e0-b345-6c86d2047c07::span:1634:1638~~d0de6a86-66d5-40e0-b345-6c86d2047c07::span:1650:1659',
#         label=False,
#         semantics=None),
#     Explanation(
#         name='LF_married_after',
#         condition="the word 'and' is between arg 1 and arg 2 and 'married' or 'marriage' is after arg 2",
#         candidate='e522e66f-ad1f-4b8b-a532-4f030a8e7a75::span:4054:4059~~e522e66f-ad1f-4b8b-a532-4f030a8e7a75::span:4085:4091',
#         label=True,
#         semantics=None),
]

In [6]:
parses = semparser.parse(explanations, return_parses=True)
print(len(parses))
print(parses[0])

2
($ROOT ($Start <START>) ($ROOT_$Start ($LF ($Label label) ($LF_$Label_ ($Bool ($BoolLit ($True true))) ($LF_$Label__$Bool ($Because if) ($Bool ($Bbox ($Box box) ($BoxId ($X x))) ($BboxToBool ($PointCompare ($Near near)) ($Bbox ($Box box) ($BoxId ($Y y)))))))) ($Stop <STOP>)))


In [7]:
print(parses[0].semantics)

('.root', ('.label', ('.bool', True), ('.call', ('.near', ('.box', ('.int', 1))), ('.box', ('.int', 0)))))


In [8]:
sem = ('.root', ('.label', ('.bool', True), ('.call', ('.near', ('.box', ('.int', 1))), ('.box', ('.int', 0)))))
sem2 = ('.root', ('.label', ('.bool', True), ('.call', ('.near_', ('.box', ('.int', 1))), ('.box', ('.int', 0)), ('.float', 30.0))))

In [15]:
original_func = semparser.grammar.evaluate(parses[0])
print(original_func)

<function <lambda> at 0x112253f50>


In [10]:
parses[0].semantics = sem2

In [17]:
new_func = semparser.grammar.evaluate(parses[0])
print(new_func)

<function <lambda> at 0x11225d5f0>


In [19]:
semparser.grammar.translate(sem)

'return 1 if call(near(Box X), Box Y) else 0'

In [13]:
# parses = semparser.parse(explanations)